In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

In [3]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [4]:
def init_params():
    W1 = np.random.randn(10,784)
    b1 = np.random.rand(10,1)
    W2 = np.random.rand(10,10)
    b2 = np.random.rand(10,1)
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(0,Z)


def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def deriv_ReLU(Z):
    return Z > 0


def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y


def back_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = Y.size
    
    one_hot_Y = one_hot(Y)
    
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_param(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [5]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_param(W1, b1, W2, b2,dW1, db1, dW2, db2,alpha)
        if (i % 10 == 0):
            print("Iteration:", i)
            print("Accuracy:", get_accuracy(get_predictions(A2),Y))
    return W1, b1, W2, b2

In [7]:
W1, b1, W2, b2 = gradient_descent(X_train,Y_train,500, 0.2)

Iteration: 0
[4 4 4 ... 4 4 4] [4 5 6 ... 2 3 9]
Accuracy: 0.08509756097560976
Iteration: 10
[6 8 6 ... 7 6 7] [4 5 6 ... 2 3 9]
Accuracy: 0.33887804878048783
Iteration: 20
[5 8 6 ... 9 6 7] [4 5 6 ... 2 3 9]
Accuracy: 0.3720731707317073
Iteration: 30
[5 8 6 ... 9 9 7] [4 5 6 ... 2 3 9]
Accuracy: 0.4086341463414634
Iteration: 40
[5 8 6 ... 9 9 7] [4 5 6 ... 2 3 9]
Accuracy: 0.4313658536585366
Iteration: 50
[5 8 6 ... 9 9 7] [4 5 6 ... 2 3 9]
Accuracy: 0.45053658536585367
Iteration: 60
[5 8 6 ... 9 9 7] [4 5 6 ... 2 3 9]
Accuracy: 0.46587804878048783
Iteration: 70
[6 8 6 ... 3 9 7] [4 5 6 ... 2 3 9]
Accuracy: 0.47573170731707315
Iteration: 80
[6 3 6 ... 9 9 7] [4 5 6 ... 2 3 9]
Accuracy: 0.46202439024390246
Iteration: 90
[6 3 6 ... 3 9 7] [4 5 6 ... 2 3 9]
Accuracy: 0.46607317073170734
Iteration: 100
[6 3 6 ... 3 9 7] [4 5 6 ... 2 3 9]
Accuracy: 0.471780487804878
Iteration: 110
[6 3 6 ... 3 9 7] [4 5 6 ... 2 3 9]
Accuracy: 0.4776829268292683
Iteration: 120
[6 3 6 ... 3 9 7] [4 5 6 ... 2